In [ ]:
!pip install pretty_midi mir_eval numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 15.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.7/90.7 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.4 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592289 sha256=d49aa2f16a3fe473650b2b8e3aa377b0ffc831662570de7f97ee538a844a409a
  Stored in directory: /root/.cache/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891ebf0be9ef07a8af49f25
  Created wheel for mir_eval: filename=mir_eval-0.7-py3-none-any.whl size=100704 sha256=a8fb8374053a3386ae2aebdb0ae33d728ab603258989859088979bfe5e6e35a9
  Stored in directory: /root/.cache/pip/wheels/3e/2f/0d/dda9c4c77a170e21356b6afa2f7d9bb078338634ba05d94e3f
Successfully built pretty_midi mir_eval
  Attempting uninstall: packa

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pretty_midi
import mir_eval
import numpy as np

def read_note_data_from_text(file_path):
    intervals = []
    pitches = []
    velocities = []
    with open(file_path, 'r') as file:
        for line in file:
            columns = line.strip().split()
            start, end, pitch, velocity = columns
            intervals.append([float(start), float(end)])
            pitches.append(int(pitch))
            velocities.append(int(velocity))
    return np.array(intervals), np.array(pitches), np.array(velocities)

def prepare_data_for_evaluation(midi_file):
    midi_data = pretty_midi.PrettyMIDI(midi_file)
    intervals = []
    pitches = []
    velocities = []
    for instrument in midi_data.instruments:
        for note in instrument.notes:
            intervals.append([note.start, note.end])
            pitches.append(note.pitch)
            velocities.append(note.velocity)
    return np.array(intervals), np.array(pitches), np.array(velocities)

def read_note_data_from_npy(file_path):
    data = np.load(file_path)
    intervals = data[:, :2]
    pitches = data[:, 2]
    return intervals, pitches

def evaluate_midi(predicted_midi_file, truth_midi_file):
    predicted_intervals, predicted_pitches, predicted_velocities = prepare_data_for_evaluation(predicted_midi_file)
    truth_intervals, truth_pitches, truth_velocities = read_note_data_from_text(truth_midi_file)

    full_precision, full_recall, full_f1_score, overlap = mir_eval.transcription_velocity.precision_recall_f1_overlap(
        truth_intervals, truth_pitches, truth_velocities,
        predicted_intervals, predicted_pitches, predicted_velocities,
        onset_tolerance=0.05, pitch_tolerance=50.0, offset_ratio=None,
        offset_min_tolerance=0.05, strict=False, velocity_tolerance=0.1, beta=1.0)

    onset_precision, onset_recall, onset_f1_score = mir_eval.transcription.onset_precision_recall_f1(
        truth_intervals, predicted_intervals, onset_tolerance=0.05, strict=False, beta=1.0)

    offset_precision, offset_recall, offset_f1_score = mir_eval.transcription.offset_precision_recall_f1(
        truth_intervals, predicted_intervals, offset_ratio=0.2, offset_min_tolerance=0.05, strict=False, beta=1.0)

    return {
        'full': (full_precision, full_recall, full_f1_score),
        'onset': (onset_precision, onset_recall, onset_f1_score),
        'offset': (offset_precision, offset_recall, offset_f1_score),
        'overlap': overlap
    }

def evaluate_folders(predicted_folder, truth_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    full_eval_path = os.path.join(output_folder, "full_evaluation_results.txt")
    onset_eval_path = os.path.join(output_folder, "onset_evaluation_results.txt")
    offset_eval_path = os.path.join(output_folder, "offset_evaluation_results.txt")
    overlap_eval_path = os.path.join(output_folder, "overlap_evaluation_results.txt")
    full_avg_path = os.path.join(output_folder, "full_evaluation_average.txt")
    onset_avg_path = os.path.join(output_folder, "onset_evaluation_average.txt")
    offset_avg_path = os.path.join(output_folder, "offset_evaluation_average.txt")
    overlap_avg_path = os.path.join(output_folder, "overlap_evaluation_average.txt")

    full_scores = []
    onset_scores = []
    offset_scores = []
    overlap_scores = []

    with open(full_eval_path, 'w') as full_file, open(onset_eval_path, 'w') as onset_file, \
         open(offset_eval_path, 'w') as offset_file, open(overlap_eval_path, 'w') as overlap_file:
        full_file.write("File, Precision, Recall, F1 Score\n")
        onset_file.write("File, Precision, Recall, F1 Score\n")
        offset_file.write("File, Precision, Recall, F1 Score\n")
        overlap_file.write("File, Overlap\n")

    for predicted_file in os.listdir(predicted_folder):
        if not predicted_file.endswith('.mid'):
            continue

        predicted_path = os.path.join(predicted_folder, predicted_file)
        truth_file = predicted_file.rsplit('.', 1)[0] + '.txt'
        truth_path = os.path.join(truth_folder, truth_file)

        if os.path.isfile(truth_path):
            try:
                results = evaluate_midi(predicted_path, truth_path)

                full_scores.append(results['full'])
                onset_scores.append(results['onset'])
                offset_scores.append(results['offset'])
                overlap_scores.append(results['overlap'])

                with open(full_eval_path, 'a') as full_file, open(onset_eval_path, 'a') as onset_file, \
                     open(offset_eval_path, 'a') as offset_file, open(overlap_eval_path, 'a') as overlap_file:
                    full_file.write(f"{predicted_file}, {results['full'][0]}, {results['full'][1]}, {results['full'][2]}\n")
                    onset_file.write(f"{predicted_file}, {results['onset'][0]}, {results['onset'][1]}, {results['onset'][2]}\n")
                    offset_file.write(f"{predicted_file}, {results['offset'][0]}, {results['offset'][1]}, {results['offset'][2]}\n")
                    overlap_file.write(f"{predicted_file}, {results['overlap']}\n")

            except ValueError as e:
                # Error
                pass

    def calculate_and_save_averages(scores, file_path):
        if scores:
            avg_precision = sum(score[0] for score in scores) / len(scores)
            avg_recall = sum(score[1] for score in scores) / len(scores)
            avg_f1 = sum(score[2] for score in scores) / len(scores)
            with open(file_path, 'w') as file:
                file.write(f"Average Precision, Average Recall, Average F1 Score, Average Velocity, Average Overlap\n")
                file.write(f"{avg_precision}, {avg_recall}, {avg_f1}\n")

    def calculate_and_save_average_overlap(scores, file_path):
        if scores:
            avg_overlap = sum(scores) / len(scores)
            with open(file_path, 'w') as file:
                file.write(f"Average Overlap\n")
                file.write(f"{avg_overlap}\n")

    calculate_and_save_averages(full_scores, full_avg_path)
    calculate_and_save_averages(onset_scores, onset_avg_path)
    calculate_and_save_averages(offset_scores, offset_avg_path)
    calculate_and_save_average_overlap(overlap_scores, overlap_avg_path)

os.chdir("/content/drive/MyDrive/Dissertation Code")
predicted_folder = "OMAPS2/evaluation/skipping-the-frame-level"
truth_folder = "OMAPS2/complete/text"

evaluate_folders(predicted_folder, truth_folder, predicted_folder)


ModuleNotFoundError: No module named 'pretty_midi'

NameError: name 'read_note_data_from_npy' is not defined